In [10]:

# pip install -e 'git+https://github.com/nmdp-bioinformatics/service-act.git#egg=1.0.1&subdirectory=client-python' 
# pip install biopython
# pip install pandas

from Bio import SeqIO
from io import StringIO
import pandas as pd

import act_client
from act_client.rest import ApiException
from act_client.api_client import ApiClient

api_client = ApiClient(host="http://act.b12x.org")
act_api = act_client.DefaultApi(api_client=api_client)


In [9]:
# Print out the results of the /act API
gfe_typing = []
with open("tutorial.fasta", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        [loc, sampleid] = record.id.split("|")
        allele_call = act_api.act_get(locus=loc, sequence=str(record.seq))
        hla = "/".join([typ.hla for typ in allele_call.typing])
        ref = "/".join([typ.hla for typ in allele_call.ihiw_ref])
        status = allele_call.typing_status.status
        feats = []
        if hasattr(allele_call.typing_status,'novel_features'):
            for feat in allele_call.typing_status.novel_features:
                feats.append("-".join([feat.term, str(feat.rank)]))
        novel_feats = ",".join(feats)
        gfe_typing.append([sampleid, allele_call.gfe, hla, ref, status, novel_feats])

gfe_df = pd.DataFrame(gfe_typing, columns=["SampleID", "GFE", "AlleleCall", "Reference","Status","Novel Features"])
print(gfe_df)

  SampleID                                         GFE  \
0    test1  HLA-Aw506-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-10   
1    test2     HLA-Aw1-5-4-5-4-9-2-4-2-3-2-2-1-1-2-1-1   

                             AlleleCall          Reference             Status  \
0                     HLA-A*01:01:01:01  HLA-A*01:01:01:01              novel   
1  HLA-A*02:01:01:01/HLA-A*02:01:01:02L  HLA-A*02:01:01:01  novel_combination   

     Novel Features  
0  five_prime_UTR-1  
1                    


In [2]:
# Print out GFE results as IMGT data format
with open("tutorial.fasta", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        [loc, tid] = record.id.split("|")
        allele_call = act_api.actformat_get(locus=loc, sequence=str(record.seq), format_type="imgt")
        print(allele_call)

ID   GFEw9269; ; ; DNA; ; HUM; 3503 BP.
XX
AC   GFEw9269;
XX
DE   GFE HLA-Aw506-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-10
XX
OS   .
OC   .
XX
CC   Typing Status: novel
CC   Novel features: five_prime_UTR-1
CC   Allele Call: HLA-A*01:01:01:01
CC   IHIW Reference: HLA-A*01:01:01:01
CC   
CC   Typed with ACT Service 0.0.4
CC   Annotated with GFE Service 1.0.8
XX
FH   Key                 Location/Qualifiers
FH
FT   source              1..3503
FT                       /organism="Homo sapiens"
FT                       /mol_type="genomic DNA"
FT                       /db_xref="taxon:9606"
FT   UTR                 1..282
FT   exon                301..373
FT                       /number=1
FT   intron              374..503
FT                       /number=1
FT   exon                504..773
FT                       /number=2
FT   intron              774..1014
FT                       /number=2
FT   exon                1015..1290
FT                       /number=3
FT   intron              1291..1869
FT  

In [5]:
# Print out GFE results as genbank data format
with open("tutorial.fasta", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        [loc, tid] = record.id.split("|")
        allele_call = act_api.actformat_get(locus=loc, sequence=str(record.seq), format_type="genbank")
        print(allele_call)

LOCUS       GFEw9269                3503 bp    DNA              PRI 01-JAN-1980
DEFINITION  GFE HLA-Aw506-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-10.
ACCESSION   GFEw9269
VERSION     GFEw9269
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
COMMENT     Typing Status: novel
            Novel features: five_prime_UTR-1
            Allele Call: HLA-A*01:01:01:01
            IHIW Reference: HLA-A*01:01:01:01
            
            Typed with ACT Service 0.0.4
            Annotated with GFE Service 1.0.8
FEATURES             Location/Qualifiers
     source          1..3503
                     /organism="Homo sapiens"
                     /mol_type="genomic DNA"
                     /db_xref="taxon:9606"
     UTR             1..282
     exon            301..373
                     /number=1
     intron          374..503
                     /number=1
     exon            504..773
                     /number=2
     intron          774..1014
                     /number=2
     exon          